# <center> Introduction to Reinforcement Learning</center>

#### Import dependencies

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import pickle
import math
import sys
import os
from Practical11_Support.gym_simple_gridworlds.helper import *
from Practical11_Support.gym_simple_gridworlds.envs.grid_env import GridEnv
from Practical11_Support.gym_simple_gridworlds.envs.grid_2dplot import *

from IPython.display import display, HTML

# The Grid World environment

Recall the grid in which our robot lives

![GridWorldExample.png](https://i.postimg.cc/5tMM5vqf/Grid-World-Example.png)

- The states $s \in \mathcal{S}$ correspond to locations in the grid. Each location has also a cell index associated to it, e.g., cell index 4 is associated to location (row=1,col=0)
- The robot can move up, down, left, or right. Actions correpond to unit increments or decrements in the specified direction.
    - Up : (-1,0)
    - Down: (1,0)
    - Left: (0,-1)
    - Right: (0, 1)
- Each action is represented by a number. Action (Up) is represented by 0, (Down) by 1, (Left) by 2 and, finally, (Right) by 3. No actions are available at a terminal state
- Discount factor $\gamma = 0.99$ (class attribute ``gamma=0.99``)
- Stochastic transition matrix (class attribute ``noise=0.2``)
- Rewards are only obtained at terminal states (class attribute ``living_reward=-0.04``)

This environment is represented with the class ``GridEnv``. **To a look at the attributes of this class, place the cursor somewhere on the class' name and hit SHIFT+TAB (local Jupyter Notebook) or hover your mouse over it (Colab). If there's a + button at the top of the popup tooltip, this means the documentation spans a few lines, click it to show the full docstring, then scroll up.**



# Activity 1. Elements of an MDP (Grid World Example)

## Learning Objectives
By the end of this activity, you will be able to:

* Describe the components of a Markov Decision Process (MDP).

* Explore state and action spaces in a grid environment.

* Understand transition dynamics and rewards.

* Observe how a policy influences agent behavior.





## Create Environment and Explore its Attributes

The noise parameter corresponds to the probability of a change of direction when an action is taken (e.g., going left/right when agent decides to move up/down)

In [ ]:
# Create a Grid World instance
grid_world = GridEnv(gamma=0.9, noise=0.2, living_reward=-0.04)

### State and Action Spaces

Let's take a look at the state and action spaces of our environment

In [ ]:
# State (or observation) space
print(grid_world.observation_space)
print(grid_world.get_states())
print()

# Action space
print(grid_world.action_space)
print(grid_world.get_actions())

Discrete(11) → there are 11 valid states in this GridWorld.

The environment encodes each state as an integer index 0 … 10.

Why 11 instead of 12 (3×4 grid)?

Because walls / blocked cells are not part of the state space.

Only usable positions are numbered.

So each number corresponds to a cell (row, col) in the grid, except walls.

✅ This is a common OpenAI Gym convention: states are represented by integers instead of (row, col) tuples.

Discrete(4) → there are 4 possible actions.

They’re encoded as integers [0, 1, 2, 3].

Typically:

0 = North (Up)

1 = South (Down)

2 = West (Left)

3 = East (Right)

(This exact mapping depends on the implementation of GridEnv, but the idea is the same.)

### Transition Function

Let's take a look at the current state transition function. Some things to keep in mind regarding the transition function:

1. Given that $\mathcal{T}: \mathcal{S} \times \mathcal{A} \times \mathcal{S} \rightarrow \mathbb{R}$, the ``state_transitions`` attribute of the class ``GridEnv`` corresponds to a 3-Dimensional numpy array of size $11\times4\times11$.
2. With a noise attribute set to 0.2, at state 5, if the agent chooses to move up, it will end up at:
    - state 2 with $80\%$ probability,
    - state 6 with $10\%$ probability, or
    - state 5 with $10\%$ probability

![GridWorldExample.png](https://i.postimg.cc/5tMM5vqf/Grid-World-Example.png)

![Screenshot from 2025-09-04 00-40-26.png](https://i.postimg.cc/fTr59Ls8/stuff.png)

In [ ]:
# at state 5 the agent takes action 0 (going up)
print(grid_world.state_transitions[5,0])

grid_world.state_transitions[5,0]

First index = state (here 5);
Second index = action (here 0, which probably means “Up/North”)

The returned row = probability distribution over next states.
[0.  0.  0.8  0.  0.  0.1  0.1  0.  0.  0.  0.]
 ↑  ↑    ↑              ↑   ↑
 s0 s1   s2             s5  s6

From state 5, action 0 leads to:

State 2 with prob 0.8 (the intended “up” move)

State 5 with prob 0.1 (blocked or slip → stays in place)

State 6 with prob 0.1 (slip to the side)

All other states → probability 0.0

General rule

For any (s, a):

𝓣⟮s , a, s'⟯ = ℙ⟮ s'| s, a⟯ = grid_world.state_transitions[s, a][s']

### Living Reward and Reward Function

Let's now take a quick look at the living reward (i.e., running cost) and reward function $\mathcal{R}: \mathcal{S} \times \mathcal{A} \rightarrow \mathbb{R}$.

1. Living reward corresponds to the attribute ``living_rewards`` of the class ``GridEnv`` and is represented as an 1-Dimensional numpy array
2. The reward function corresponds to the attribute ``rewards`` of the class ``GridEnv`` and is also represented as a 2-Dimensional numpy array of size $11\times4$

In [ ]:
# Living rewards
print("Living rewards for all states:\n{}\n".format(grid_world.immediate_rewards))

This is a 1-D array of size = number of states (11).

Each entry = reward for being in that state at a step (before considering action outcomes).

In [ ]:
# Reward function, i.e., the expected reward for taking action a at state s
print("Reward function for all state-action pairs:\n{}\n".format(grid_world.rewards))
print("The expected reward at state 5 if agent chooses to move right is: {}".format(grid_world.rewards[5,3]))

This is a 2-D array of shape (n_states, n_actions).

Entry rewards[s, a] = expected reward if you take action a in state s.

### **TODO** (Flux Quiz 2): what is the expected reward at state 2 if the agent chooses to move right?

### Policy

Let's see the path and total reward of an agent moving on our grid world according to the following policy $\pi$

![example_policy.png](https://i.postimg.cc/pLjHnkj0/example-policy.png)

In [ ]:
# We represent this policy as a 2-Dimensional numpy array
# This is a fixed, hand-coded policy
policy_matrix = np.array([[3,      3,  3,  -1],
                          [0, np.nan,  0,  -1],
                          [0,      2,  0,   2]])

In [ ]:
print(grid_world.grid)

Let's now apply this fixed, hand-coded policy and observe the agent's behavior (blue dot in the figure shown below).

In Grid World, terminal states are the goal (reward +1) and the trap (reward –1).

Once the agent reaches one of those, the environment sets done=True.

That’s what stops the loop.

In [ ]:
# Create a Grid World instance
grid_world = GridEnv(gamma=0.9, noise=0.2, living_reward=-0.04)
s_x, s_y = get_state_to_plot(grid_world)

# We can visualize our grid world using the render() function
fig, ax = grid_world.render()
agent, = ax.plot([], [], 'o', color='b', linewidth=6)
reward_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

done = False
cumulative_reward = 0
cur_state = grid_world.cur_state # where the agent starts
path_to_plot = []

while not done:
    _, cur_reward, done, _ = grid_world.step(int(policy_matrix[cur_state[0], cur_state[1]]))
    cur_state = grid_world.cur_state
    n_x, n_y = get_state_to_plot(grid_world)
    cumulative_reward += cur_reward
    path_to_plot.append([cumulative_reward, n_x, n_y])

def init():
    agent.set_data([s_x + 0.5], [s_y + 0.5])
    reward_text.set_text('')
    return agent, reward_text

def animate(i):
    if i < len(path_to_plot):
        r, n_x, n_y = path_to_plot[i]
        agent.set_data([n_x + 0.5], [n_y + 0.5])
        reward_text.set_text('Cumulative reward: %.2f' % r)
    return agent, reward_text

ani = animation.FuncAnimation(fig, animate, frames=len(path_to_plot), blit=False, interval=500, init_func=init,
                              repeat=False)

plt.close('all')
display(HTML(f"<div align=\"center\">{ani.to_jshtml()}</div>"))

In [ ]:
### Helper Function ###
def policy_evaluation(grid_env, policy, plot=False, threshold=0.00001):

    """
    This function computes the value function for a policy pi in a given environment grid_env.

    :param grid_env (GridEnv): MDP environment
    :param policy (dict - stochastic form): Policy being evaluated
    :return: (dict) State-values for all non-terminal states
    """

    # Obtain list of all states in environment
    v = {s: 0.0 for s in grid_env.get_states()}
    theta = threshold
    delta = 1000

    while delta > theta:
        delta = 0.0
        # For all states
        for s in v.keys():

            old_v = v[s]
            new_v = 0

            # For all actions
            for a, probability_a in policy[s].items():
                discounted_v = 0

                # For all states that are reachable from s with action a
                for s_next in grid_env.get_states():
                    #TODO 1: Compute discounted sum of state values for all successor states ---------
                    discounted_v += 0
                    #ENDTODO -------------------------------------------------------------------------

                #TODO 2: Compute expectation over all actions ------------------------------------
                new_v += 0
                #ENDTODO -------------------------------------------------------------------------

            v[s] = new_v
            delta = max(delta, np.abs(old_v - new_v))

    if plot:
        plot_value_function(grid_env, v)

    return v

# Activity 2 Model-Free RL:Q-Learning
What if the Transition and Reward Function are Unknown?

How to evaluate a policy without a model.

Let's now find an *approximately* optimal policy using the off-policy control method Q-learning.

To help during the learning, we have added a lambda function that iteratively decreases epsilon. Our agent will strongly explore the environment at first to then swicth into exploitation mode

#  The Grid World environment

Recall the grid in which our robot lives

![GridWorldExample.png](https://i.postimg.cc/5tMM5vqf/Grid-World-Example.png)

- The states $s \in \mathcal{S}$ correspond to locations in the grid. Each location has also a cell index associated to it, e.g., cell index 4 is associated to location (row=1,col=0)
- The robot can move up, down, left, or right. Actions correpond to unit increments or decrements in the specified direction.
    - Up : (-1,0)
    - Down: (1,0)
    - Left: (0,-1)
    - Right: (0, 1)
- Each action is represented by a number. Action (Up) is represented by 0, (Down) by 1, (Left) by 2 and, finally, (Right) by 3. No actions are available at a terminal state
- Discount factor $\gamma = 0.99$ (class attribute ``gamma=0.99``)
- Stochastic transition matrix (class attribute ``noise=0.2``)
- Rewards are only obtained at terminal states (class attribute ``living_reward=-0.04``)

Here is our implementation of the q-learning algorithm shown below

![q-learning.png](https://i.postimg.cc/8z70Yv5C/q-learning.png)

**Complete the missing steps**:
- Choose an action using an $\epsilon$-greedy policy (use the function ``get_egreedy_action(.)`` we tested in Section 3.1)
- Update our q-function using a greedy (max) policy (use ``q_function[cur_state][action]`` to index our q-function)

## TODO (Flux Quiz 3) Complete the missing steps:
- Choose an action using an $\epsilon$-greedy policy (use the function ``get_egreedy_action(.)``)
- Update our q-function using a greedy (max) policy (use ``q_function[cur_state][action]`` to index our q-function)

**Keep in Mind**: Correspondance between the mathematical notation and implemented code

|                 |                            |                  |
| :-------------- | -------------------------: | ---------------: |
|                 | **Variable/Attribute**     | **Type**         |
| $\epsilon$      | `epsilon_by_episode`       | `float`          |
| $\alpha$        | `alpha`                    | `float`          |
| $\gamma$        | `grid_world.gamma`         | `float`          |
| $\hat{q}(s, a)$ | `q_function[idx_s][idx_a]` | `dict` of `dict` |
| $s$             | `cur_state`                | `int`            |
| $r$             | `reward`                   | `int`            |
| $s^{\prime}$    | `next_state`               | `int`            |

### Step 1 initialize the Q-table with values of 0.

In [ ]:
grid_world = GridEnv(gamma=0.9, noise=0.2, living_reward=-0.04)
states = grid_world.get_states()
actions = grid_world.get_actions()
# initialize the Q-table with values of 0.
q_function = defaultdict(lambda: defaultdict(float))

for s in states:
    for a in actions:
        q_function[s][a] = 0.0

def print_q_table(q_function, actions):
    print("State".ljust(12) + "".join([f"A{a}".rjust(12) for a in actions]))
    print("-" * (12 + 12*len(actions)))
    for s, q_vals in q_function.items():
        row = str(s).ljust(12) + "".join([f"{q_vals[a]:10.3f}  " for a in actions])
        print(row)

print("\n=== Initial Q-Table (all zeros) ===")
print_q_table(q_function, actions)

### Step 2: Choose an action using the epsilon-greedy strategy


#### Define $\epsilon$-Greedy Policy
Epsilon-Greedy is the training policy that handles the exploration/exploitation trade-off.

The idea with Epsilon Greedy:

- With probability **1 - ɛ** : **we do exploitation** (aka our agent selects the action with the highest state-action pair value).

- With probability **ɛ**: **we do exploration** (trying random action).

And as the training goes, we progressively reduce the epsilon value since we will need less and less exploration and more exploitation.


$\epsilon$-greedy policy is formally defined as:

$$
\begin{aligned}
    \pi(a|s) =
    \begin{cases}
        1 - \epsilon + \frac{\epsilon}{|\mathcal{A}|},&  \text{if } a^* = \arg\max_{a \in \mathcal{A}} q_\pi(s,a)\\
        \frac{\epsilon}{|\mathcal{A}|}, & \text{otherwise}
    \end{cases}
\end{aligned}
$$

Let's see how the agent behaves when it follows an $\epsilon$-greedy policy.

In [ ]:
def get_egreedy_action(grid_env, state, q_value, epsilon):
    """
    Select action to execute at a given state under an epsilon-greedy policy
    :param grid_env (GridEnv): Grid world environment
    :param state (int): Location in grid for which next action is going to be choosen
    :param q_value (dict): Action-value function
    :param epsilon (float): Randomness threshold used to choose action
    """

    rand_n = np.random.random()
    if rand_n <= epsilon:
        # Explore: pick a random action
        return grid_env.action_space.sample()
    else:
        # Exploit: pick the best action
        actions = list(q_value[state].keys())
        return actions[np.argmax(list(q_value[state].values()))]

At the beginning of the training, the probability of doing exploration will be huge since ɛ is very high, so most of the time, we’ll explore.

But as the training goes on, and consequently our Q-table gets better and better in its estimations, we progressively reduce the epsilon value since we will need less and less exploration and more exploitation.

epsilon_by_episode (the lambda function)

- Defines how ε changes with episode index:
	​
- This is an exponential decay schedule.

- Early episodes: ε is close to 1.0 (explore).

- Later episodes: ε gets closer to 0.001 (exploit).

In [ ]:
min_epsilon=0.001
max_epsilon=1.0
epsilon_decay = 80.0
epsilon_by_episode = lambda ep_idx: (min_epsilon
         + (max_epsilon - min_epsilon)
         * math.exp (-1 * ep_idx/epsilon_decay))

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))

ax.plot([epsilon_by_episode(i) for i in range(500)])
ax.set_xlabel("Num. episodes")
ax.set_ylabel("Epsilon")

#### Define the greedy policy
Remember we have two policies since Q-Learning is an **off-policy** algorithm. This means we're using a **different policy for acting and updating the value function**.

- Epsilon greedy policy (acting policy)
- Greedy policy (updating policy)

Greedy policy will also be the final policy we'll have when the Q-learning agent will be trained. The greedy policy is used to select an action from the Q-table.

In [ ]:
def greedy_policy(Qtable, state):
  # Exploitation: take the action with the highest state, action value
  action = np.argmax(Qtable[state])

  return action

In [ ]:
def q_learning(grid_env, alpha=0.1, min_epsilon=0.01, max_epsilon=1.0,
               epsilon_decay = 80.0, n_episodes=500):
    """
    This function computes an approximately optimal policy using q-learning

    :param grid_env (GridEnv): MDP environment
    :param alpha (float): step-size
    :param epsilon (float): value used during e-greedy action selection
    :return: (dict) State-values for all non-terminal states
    """

    # This lambda function iteratively decreases epsilon
    epsilon_by_episode = lambda ep_idx: min_epsilon + (max_epsilon - min_epsilon) * math.exp (-1 * ep_idx/epsilon_decay)

    # Obtain list of all states in environment
    states = grid_env.get_states()
    actions = grid_env.get_actions()
    q_function = defaultdict(lambda: defaultdict(float))

    # Initialize q_function arbitrarily
    for s in states:
        for a in actions:
            q_function[s][a] = 0

    episode_returns = []
    for i_episode in range(1, n_episodes+1):
        cur_state = grid_env.reset()
        done = False
        epsilon = epsilon_by_episode(i_episode)
        G = 0.0
        while not done:
            #TODO 1: Complete off-policy action selection (e-greedy)-----------
            #Choose the action using epsilon greedy policy
            action = None
            #ENDTODO ----------------------------------------------------------

            #Take action At and observe Rt+1 and St+1
            #Take the action (a) and observe the outcome state(s') and reward (r)
            next_state, reward, done,_ = grid_env.step(action)
            q_next_state = list(q_function[next_state].values())

            #TODO 2: Complete update of q-function -----------------------------
            # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
            q_function[cur_state][action] += 0
            #ENDTODO ------------------------------------------------------------
            G += reward
            cur_state=next_state
        episode_returns.append(G)

    return q_function, decode_policy(grid_env, q_function), episode_returns

In [ ]:
grid_world = GridEnv(gamma=0.9, noise=0.2, living_reward=-0.04)
q_function, q_learning_policy, _ = q_learning(grid_world)

plot_policy(grid_world, q_learning_policy)
print(q_learning_policy)

# Compute value function for q_learning policy
q_policy_state_values = policy_evaluation(grid_world, encode_policy(grid_world, q_learning_policy))
plot_value_function(grid_world, q_policy_state_values)

Let's see what our Q-Learning table looks like now 👀

In [ ]:
import numpy as np

states  = grid_world.get_states()   # e.g., [(0,0),(0,1),...]
actions = grid_world.get_actions()  # e.g., [0,1,2,3]

state_to_idx  = {s: i for i, s in enumerate(states)}
action_to_idx = {a: j for j, a in enumerate(actions)}

Q = np.zeros((len(states), len(actions)))
for s in states:
    for a in actions:
        Q[state_to_idx[s], action_to_idx[a]] = q_function[s][a]

print(Q)  # rows = states (in the order of grid_env.get_states()), cols = actions

### An ablation of $\epsilon$ on $\epsilon$-greedy policy

Train Q-learning with three ε settings and compare policies/returns:
- (A) ε = 0.9 constant, dubbed `high`
- (B) ε = 0.1 constant, dubbed `low`
- (C) ε decays from 1.0 → 0.01, dubbed `decay`

Goal: Compare policies and returns for three ε schedules:

In [ ]:
def run_q_with_schedule(schedule: str):
    env = GridEnv(gamma=0.99, noise=0.2, living_reward=-0.04)
    if schedule == "high":
        return q_learning(env, n_episodes=500, min_epsilon=0.9, max_epsilon=0.9, epsilon_decay=1.0)
    if schedule == "low":
        return q_learning(env, n_episodes=500, min_epsilon=0.1, max_epsilon=0.1, epsilon_decay=1.0)
    # default decay
    return q_learning(env, n_episodes=500, min_epsilon=0.01, max_epsilon=1.0, epsilon_decay=80.0)

Q_high, _, ret_high   = run_q_with_schedule("high")
Q_low, _, ret_low     = run_q_with_schedule("low")
Q_decay, _, ret_decay = run_q_with_schedule("decay")

grid_env = GridEnv(gamma=0.9, noise=0.2, living_reward=-0.04)

for (Q_, name) in zip([Q_high, Q_low, Q_decay], ["High", "Low", "Decay"]):
    pol = decode_policy(grid_env, Q_)
    print(f"{name} ε policy:")
    plot_policy(grid_env, pol)
    plt.show()

# Simple running-mean curves (optional)
def running_mean(x, N=20):
    if len(x) < N:
        return np.array(x)
    return np.convolve(x, np.ones(N)/N, mode='valid')

plt.figure(); plt.plot(running_mean(ret_high)); plt.title('Running mean returns – High ε')
plt.figure(); plt.plot(running_mean(ret_low)); plt.title('Running mean returns – Low ε')
plt.figure(); plt.plot(running_mean(ret_decay)); plt.title('Running mean returns – Decay ε')
plt.show()

1. High ε (0.9 constant)

- Curve stays around low/negative returns with lots of fluctuation.

- Explanation: the agent is almost always exploring randomly, so it never really settles on a good policy. That’s why it looks noisy and hovers near baseline performance.

2. Low ε (0.1 constant)

- Curve quickly climbs to higher positive returns and then oscillates around a stable level.

- Explanation: the agent exploits early, so it finds a decent policy fast. But with little exploration, it may converge to a suboptimal but consistent policy.

3. Decay ε (1.0 → 0.01)

- Curve starts poor (random exploration), then steadily improves, and finally stabilizes near high returns.

- Explanation: the agent explores widely at first, then gradually exploits, which balances learning and convergence. This is typically the best long-term strategy.